## Extra topics: Docker


fisa (@fisadev, fisadev@gmail.com)

https://github.com/fisadev/python-basic-course

# Class 6 overview

Topic: docker

- Some Linux kernel basic ideas
- The need for isolation
- Life before docker
    - Virtualization
    - chroot, "manual" LXC, etc
- Docker: general idea
- Docker vs the other tools
- More on images
    - Dockerfile basics
    - Building
    - Listing images
    - Layers, cache
    - Registries
- More on containers
    - Running, creating, starting
    - Stopping, deleting
    - Listing
    - Running with a different command
- Persistent filesystem
- Orchestration
    - Docker compose
    - Kubernetes

## Some Linux kernel basic ideas

We won't go into details, but we should know that:

- Kernel vs other stuff on top: a "linux system" is a kernel plus lots of programs and modules that we run on top of it. Distros decide which stuff, not the kernel.
- Isolation features: the kernel can isolate processes from each other in several ways.

## The need for isolation

- Sometimes you need to run stuff that you don't trust
- Sometimes you need to share resources between stuff that can't trust each other
- Sometimes you need to share resources between stuff that isn't compatible with each other

Most common examples:

- Hosting services running code from its users
- Company with areas sharing servers
- Different systems running in the same servers

## Life before docker

How did people solve those issues?

- Virtualization: host OS, virtual machines isolated from each other, emulating HW to run their own OSs, etc.
- Chroot, manual LXC, etc: ability to isolate processes inside the same OS, but very hard to use, automate, etc.

## Docker: general idea

Basically a simpler way of isolating, distributing and running apps. And making the environments **reproducible**.

## Docker: general idea

How?

1. Create a **Dockerfile**: a recipe of how to create the environment to run some specific process (app)
2. Compile (build) it creating an **image**: a photo of the program and its environment, packaged with all its requirements, ready to run
3. Launch **containers** using that image: running many instances of the program, isolated between each other and the rest of the OS

![](./docker_1.svg)

![](./docker_2.svg)

## Docker: general idea

And two important ideas:

- Each container should run **one thing** (process). Example: one container with the web app, another with the db server, etc.
- Containers are **disposable**: they don't store information, they can be killed and replaced, scaled up or down. Persisten info is stored outside (more on this later) 

## Docker: general idea

- Quite easy to use

- It uses LXC and plenty of other kernel features to isolate the processes
- They all run under the same host linux kernel
- So it doesn't need to emulate hardware, or run many OSs!
    - Super light in resources usage

- It provides tools to monitor and manage running containers, to interconnect them, etc
- It provides tools to share and reuse images: reproducible environments! (99%)


## Docker vs other tools

Vs virtualization:
- Way less resources usage: no repeated OSs, no HW emulation, etc.
- Easier to share images (both in work and in size).
- Slightly less secure, but secure enough for most scenarios (and there are solutions when it's not).

Vs manual usage of LXC and the rest of the kernel stuff:
- Way, way easier.
- Less freedom, but works for 95% of scenarios.

## More on images: Dockerfile

Docker file basics:

- It begins from another image (for example: "begin with a clean Ubuntu 16.04")
- It runs steps to prepare the env for our program (example: install dependencies, copy files)
- It specifies how the program will be ran inside

## More on images: Dockerfile

Example Dockerfile:

```Dockerfile
FROM python:3.6

# the dir for our website stuff
RUN mkdir /myweb
WORKDIR /myweb

# copy requirements file into the image, 
# and install them
COPY requirements.txt /myweb/
RUN pip install -r requirements.txt

# copy our website code
COPY web.py /myweb/

# to run our website, we need this env variable
ENV FLASK_APP web.py
# and this is how the website runs
CMD ["flask", "run", "--host=0.0.0.0"]
```

## More on images: Dockerfile

We can now build the image using our dockerfile!

(go to the folder where the Dockerfile is)

```docker build -t my_web .```

If we list the images in our machine, we can see it:

```docker images```

Remember: our web is **not** running. We just crated the image, which is a "mold" for containers.

## More on images: Dockerfile

You can also remove images with:

```docker rmi my_web```

And images can have tags, to name different versions (like `python:3.6`, `python:3.5`, etc)

## More on images: Layers, cache

If we build, and then build again, it does nothing. Why??

Docker remembers that it already ran this Dockerfile, and knows the result. It has a cache.
We can force a re run with:

```docker build --no-cache -t my_web .```

But the cache is way smarter than just that. Lets build, then change one line, and build again...

## More on images: Layers, cache

Docker caches the result of each **line** of our dockerfile, and runs only the lines it needs to re run.

This results are called **Layers**.

Each layer is like a git commit, storing the photo of the stuff at that point. And as commits, each layer points to its "parent" layer.

An image is **a collection of layers** with a name.

## More on images: Layers, cache

Also: we said `FROM python:3.6`. Where does that image come from?? How did we get its layers to build ours on top??

The answer: **Registries**.

Docker can pull and push images (collections of layers, with a name) to and from registries. Registries are repositories of images.

By default, docker knows of "The Registry", the public one. But you can create our own private one, and add it.

So `python:3.6` is just an image that someone created and published. Its Dockerfile is public too. Docker automatically downloaded it when we needed it.

## More on containers: running

So, we have our image ready. How do we run our website?

Simplest option:

```docker run --rm -t -i -p 5001:5000 my_web```

This means: docker, run a container from the image `my_web`, and attack my terminal to it (`-t -i`). Also, remove the container when the web is no longer running (`--rm`). And map the port `5001` of my machine, to the port `5000` of the container (which is kind of a different machine).

## More on containers: creating and starting

It's running, but it is tied to our terminal. That's useful for development, but not for a server (we want to run it in background, etc).

Instead, we can just create and launch the containers. And also, we can give each container a **name**:

```docker create --name web1 -p 5001:5000 my_web```

```docker start web1```

This means: hey docker, create a container named `web1` with the image `my_web`. Also, map the port, as before. And then: docker, start that container!

And that's it! Our web is running! We can access it at `http://localhost:5001`.

## More on containers: creating and starting

But there's more: we can have as many as we want!

```docker create --name web2 -p 5002:5000 my_web```

```docker create --name web3 -p 5003:5000 my_web```

And launch them at once:

```docker start web2 web3```

## More on containers: stopping, deleting

We can stop them all:

```docker stop web1 web2 web3```

And we can re-start them whenever we want, or just delete them:

```docker rm web1 web2 web3```

## More on containers: listing

We can list running containers with:

```docker ps```

But a way more useful tool is [Ctop](https://github.com/bcicen/ctop) (not easy to install, but very easy to use)

## More on containers: running with a different command

Our image specifies the command that will be ran in the containers:

```Dockerfile
# and this is how the website runs
CMD ["flask", "run", "--host=0.0.0.0"]
```

But we can run a container from that image specifying a different command. For example:

```docker run --rm -t -i my_web ls -la /myweb```

This will create a container, run `ls -la /myweb`. The container is closed when the command finishes (in the web case, it doesn't finish unless we kill it).

This is useful for debug, we can just open a terminal inside a container:

```docker run --rm -t -i my_web bash```

This is **SUPER USEFUL**. Think of it: creates a full new environment with all installed, we can do whatever we want, and the env is destroyed when we exit it. Completely isolated from our machine. And with just one command!

Free infinite dev break-all-you-want envs.

## Persistent filesystems

Containers are disposable. So what if we want to store permanent information??

We can map a folder in our machine, to a folder inside the container. That way, anything the container stores in that folder, is permanent and visible from the outside:

```docker run --rm -t -i -p 5001:5000 -v /home/fisa/devel/python-basic-course/docker_example/data/:/data my_web```

Or the same in the create command:

```docker create --name web1 -p 5001:5000 -v /home/fisa/devel/python-basic-course/docker_example/data/:/data my_web```


On both we added `-v /home/fisa/devel/python-basic-course/docker_example/data/:/data`, wich means "hey docker, when the container uses `/data`, use instead the real persisten folder `/home/fisa/devel/python-basic-course/docker_example/data/`.

We can even map multiple containers to that same folder!

## Orchestration

Ok, we have several dockerfiles ready, for our different apps and services. We need to have all of them running, how do we do that?

The answer: **Orchestration** tools.

These tools are designed to manage "fleets" of containers. I'll mention two: Docker compose and Kubernetes.

## Docker compose

- It's the easier one, but the one with the fewer features. Super basic.
- Only allows to build and launch our fleet of containers in a specific order, and to stop them.
- All is defined in a single .yml file

## Docker compose: example

docker-compose.yml

```yaml
version: "2"
services:
    web:
      hostname: web
      build:
          context: .
      depends_on:
        - worker

    worker:
      build: 
        context: ./worker/ 
```

## Docker compose: example

In this example, we have a web app, that also needs a different process which is a worker app. They have different Dockerfiles (the worker one is in a `worker` subfolder).

We can start all of them together with:

```docker-compose -f docker-compose.yml up```

And we can even specify how many containers to launch for each service!:

```docker-compose -f docker-compose.yml up --scale web=3 worker=2```

We can then stop everything with:

```docker-compose -f docker-compose.yml stop```

## Kubernetes

- Similar to docker compose, but waaaay more full of features
- Not only building, running and stopping, but a lot of other stuff:
    - scaling, and even auto scaling
    - load balancing
    - health monitoring
    - rollouts and rollbacks
    - and many more...
- Developed by google, used to run huge infrastructures
- Harder to start using it, but will scale up